<a href="https://colab.research.google.com/github/erikapaceep/NLP/blob/main/sentence_vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Sentence Vec

In [43]:
text = 'hello word what a great time to be alive'

In [ ]:
!pip install transformers

In [4]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from transformers import AutoTokenizer, AutoModel 
import torch

In [6]:
# initalize the model and the tokenizer
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'

In [8]:
# initialize our tokenizer model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [44]:
# tokenize our text using our tokenizer
tokens = tokenizer.encode_plus(text, max_length=128,
                               truncation=True,
                               padding='max_length',
                               return_tensors='pt')

In [42]:
tokens

{'input_ids': tensor([[ 101, 7592, 2773, 2054, 1037, 2307, 2051, 2000, 2022, 4142, 1010, 7632,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0,

In [13]:
# now that the input are tokenized we can pass it through our model
output = model(**tokens)

In [ ]:
output
# Extracting the last hidden state: the model output  and inside that we have this tensor called the last hidden state

In [ ]:
# this is the last hidden state
output[0]

In [18]:
# this is also the last hidden state
embeddings = output.last_hidden_state
embeddings

tensor([[[ 0.1143, -0.2471,  1.6116,  ...,  0.4679, -0.2416,  0.0699],
         [ 0.6115,  0.1260,  1.6738,  ...,  0.5420, -0.0708,  0.0985],
         [ 0.4344, -0.0689,  1.4304,  ...,  0.3937, -0.1738, -0.1908],
         ...,
         [ 0.3213, -0.0943,  1.1681,  ...,  0.3382, -0.3864,  0.0301],
         [ 0.2114, -0.1746,  1.1498,  ...,  0.3368, -0.4444, -0.0992],
         [ 0.2228, -0.2167,  1.2706,  ...,  0.3394, -0.4571, -0.1503]]],
       grad_fn=<NativeLayerNormBackward0>)

In [20]:
embeddings.shape

torch.Size([1, 128, 768])

Now we need to perform a mean pooling operation in order to create a single vector encoding and convert these form the current embedddings to sentence embedding all vector

In [22]:
# convert the embeddings from BERT into sentence vectors using a mean pooling operation:
# to do this we need to multiply our embedding tensor by the attention mask value so that we ignore the non teal token (that correspond to 0 the attention mask tensor)

attention_mask = tokens['attention_mask']
attention_mask.shape # we have one value for each one of our input

torch.Size([1, 128])

In [30]:
#we want an attention mask to be the same size as the embedding shape
mask = attention_mask.unsqueeze(-1).expand(embeddings.shape).float()

In [39]:
# Now that attention mask is the right shape we jsut need to multiply the embedding by the attention mask
masked_embeddings = embeddings * mask
masked_embeddings[0]

tensor([[ 0.1143, -0.2471,  1.6116,  ...,  0.4679, -0.2416,  0.0699],
        [ 0.6115,  0.1260,  1.6738,  ...,  0.5420, -0.0708,  0.0985],
        [ 0.4344, -0.0689,  1.4304,  ...,  0.3937, -0.1738, -0.1908],
        ...,
        [ 0.0000, -0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
        [ 0.0000, -0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000]],
       grad_fn=<SelectBackward0>)

In [38]:
# if we compare this wtih respect to what embeddings was before - the masked embeddings maked the last values 
embeddings

tensor([[[ 0.1143, -0.2471,  1.6116,  ...,  0.4679, -0.2416,  0.0699],
         [ 0.6115,  0.1260,  1.6738,  ...,  0.5420, -0.0708,  0.0985],
         [ 0.4344, -0.0689,  1.4304,  ...,  0.3937, -0.1738, -0.1908],
         ...,
         [ 0.3213, -0.0943,  1.1681,  ...,  0.3382, -0.3864,  0.0301],
         [ 0.2114, -0.1746,  1.1498,  ...,  0.3368, -0.4444, -0.0992],
         [ 0.2228, -0.2167,  1.2706,  ...,  0.3394, -0.4571, -0.1503]]],
       grad_fn=<NativeLayerNormBackward0>)

In [46]:
# now we need to sum the embeddings
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([1, 768])

In [55]:
counts = torch.clamp(mask.sum(1), min=1e-9)
counts.shape

torch.Size([1, 768])

In [ ]:
meaned = torch.mean(masked_embeddings, 1)
meaned

In [ ]:
# this is our sentence vector
mean_pooles = summed/counts
mean_pooles